In [418]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Masking, LSTM, Dense, concatenate, Dropout
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.layers import LSTM, Dense, Masking, Input, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from sklearn.base import BaseEstimator, TransformerMixin

#### Load data

In [419]:
SPECIAL_VALUE = -1.0

In [420]:
input_vector = pd.read_csv('../../data/data_input_vector.csv')
input_scalar = pd.read_csv('../../data/data_input_scalar.csv')
input_general = pd.read_csv('../../data/data_input_general_Info.csv')
output = pd.read_csv('../../data/data_output.csv')

#### Data Preprocessing of data_input_general_info.csv

In [421]:
# DATA PREPARATION

# Get max sequence length
max_seq_len_info = 1

# Drop unneeded columns
input_general = input_general.drop(columns=["VAN", "Index", "absolute_timestamp", 'stat_zeitpunkt_test'], axis=1)

# Replace 'BMW' with 'D'
input_general['fzg_verkaufsland_kfzint_kez'] = input_general['fzg_verkaufsland_kfzint_kez'].replace('BMW', 'D')

# Convert dates to timestamps
input_general['fzg_produktionsdatum'] = pd.to_datetime(input_general['fzg_produktionsdatum']).astype('int64')

# OMIT OUTLIER HANDLING

# PREPROCESSING
numerical_features = input_general.select_dtypes(include=['int64', 'float64']).columns
categorical_features = input_general.select_dtypes(include=['object']).columns

numerical_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

X_general = preprocessor.fit_transform(input_general)
X_general = X_general.toarray()  # Convert sparse matrix to dense matrix

dimension_info = X_general.shape[1]

#### Data preprocessing of data_input_scalar.csv

In [422]:
# DATA PREPARATION

# Drop unneeded columns
input_scalar = input_scalar.drop(columns=["Unnamed: 0", "absolute_timestamp"], axis=1)

# Drop rows with NaN values
input_scalar = input_scalar.dropna()

# Set auslesedatum relative to fzg_produktionsdatum (TODO check compatibility with resampling)
#input_scalar['auslesedatum'] = pd.to_datetime(input_scalar['auslesedatum']).astype('int64')
##input_general_copy = pd.read_csv('../../data/data_input_general_Info.csv')
#input_general_copy['fzg_produktionsdatum'] = pd.to_datetime(input_general_copy['fzg_produktionsdatum']).astype('int64')
#fzg_produktionsdatum_dict = input_general_copy.set_index('VAN')['fzg_produktionsdatum'].to_dict()
#input_scalar['auslesedatum'] = input_scalar.apply(lambda row: row['auslesedatum'] - fzg_produktionsdatum_dict[row['VAN']], axis=1)

# OUTLIER HANDLING

class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=3.0):
        self.threshold = threshold
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_num = X.drop(columns=['auslesedatum'], axis=1).select_dtypes(include=['int64', 'float64'])
        z_scores = np.abs((X_num - X_num.mean()) / X_num.std())
        mask = (z_scores < self.threshold).all(axis=1)
        
        # Debug: Print the number of rows before and after outlier removal
        print("Rows before outlier removal:", X.shape[0])
        print("Rows after outlier removal:", X[mask].shape[0])
        
        return X[mask]
    
outlier_remover = OutlierRemover(threshold=1.96)
input_scalar = outlier_remover.fit_transform(input_scalar)

# RESAMPLING (currently disabled)

# Convert dates to datetime
input_scalar['auslesedatum'] = pd.to_datetime(input_scalar['auslesedatum'])

# Sort values - this step is crucial for resampling
input_scalar = input_scalar.sort_values(by=['VAN', 'auslesedatum'])

# Group by 'VAN' and apply the resampling function
#input_scalar = input_scalar.groupby('VAN').resample('ME', on='auslesedatum').mean().reset_index()

# Convert dates to timestamps
input_scalar['auslesedatum'] = pd.to_datetime(input_scalar['auslesedatum']).astype('int64')

# PREPROCESSING

numerical_features = input_scalar.select_dtypes(include=['int64', 'float64']).columns

# Sort by VAN and auslesedatum
input_scalar = input_scalar.sort_values(by=['VAN', 'auslesedatum'])

# Create 3D array with shape (num_vehicles, max_seq_len, dimension)
num_vehicles = output['VAN'].nunique()
max_seq_len_scalar = input_scalar.groupby('VAN').size().max()
dimension_scalar = len(numerical_features)
X_scalar = np.full((num_vehicles, max_seq_len_scalar, dimension_scalar), fill_value=SPECIAL_VALUE, dtype=np.float64)

# Scaling + Encoding
scaler = MinMaxScaler()
input_scalar[numerical_features] = scaler.fit_transform(input_scalar[numerical_features])

for van in input_scalar['VAN'].unique():
    van_data = input_scalar[input_scalar['VAN'] == van]
    input_sequence = van_data.drop(columns=['VAN']).values
    index = output.index[output['VAN'] == van].to_list()[0]
    seq_len = input_sequence.shape[0]
    X_scalar[index, 0:seq_len, :] = input_sequence

Rows before outlier removal: 167820
Rows after outlier removal: 105645


#### Data preprocessing of data_input_vector.csv

In [423]:
# Drop unneeded columns
exclude_columns = ["Unnamed: 0", "absolute_timestamp", "BAUREIHE", 
                    "stat_zeit_temp_total_12_wert", "stat_zeit_temp_total_13_wert", 
                    "stat_zeit_temp_total_14_wert", "stat_hv_spannung_wert"]
input_vector = input_vector.drop(columns=exclude_columns, axis=1)

# Drop rows with NAN values
input_vector = input_vector.dropna()

# Convert dates to timestamps
input_vector['fzg_produktionsdatum'] = pd.to_datetime(input_vector['fzg_produktionsdatum']).astype('int64')

# Set auslesedatum relative to fzg_produktionsdatum (TODO check compatibility with resampling)
#input_vector['auslesedatum'] = pd.to_datetime(input_vector['auslesedatum']).astype(int)
#input_vector['auslesedatum'] = input_vector['auslesedatum'] - input_vector['fzg_produktionsdatum']

# OUTLIER HANDLING

class IQRBasedOutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5):
        self.factor = factor
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_num = X.drop(columns=['auslesedatum'], axis=1).select_dtypes(include=['int64', 'float64'])
        Q1 = X_num.quantile(0.25)
        Q3 = X_num.quantile(0.75)
        IQR = Q3 - Q1
        mask = ~((X_num < (Q1 - self.factor * IQR)) | (X_num > (Q3 + self.factor * IQR))).any(axis=1)
        
        # Debug: Print the number of rows before and after outlier removal
        print("Rows before outlier removal:", X.shape[0])
        print("Rows after outlier removal:", X[mask].shape[0])
        
        return X[mask]

outlier_remover = IQRBasedOutlierRemover(factor=20)
input_vector = outlier_remover.fit_transform(input_vector)

# RESAMPLING (currently disabled)

# Convert dates to datetime
input_vector['auslesedatum'] = pd.to_datetime(input_vector['auslesedatum'])

# Sort values - this step is crucial for resampling
input_vector = input_vector.sort_values(by=['VAN', 'auslesedatum'])

# Group by 'VAN' and apply the resampling function
#input_vector = input_vector.groupby('VAN').resample('ME', on='auslesedatum').mean().reset_index()

# Convert dates to timestamps
input_vector['auslesedatum'] = pd.to_datetime(input_vector['auslesedatum']).astype('int64')

# PREPROCESSING

numerical_features = input_vector.select_dtypes(include=['int64', 'float64']).columns

# Sort by VAN and auslesedatum
input_vector = input_vector.sort_values(by=['VAN', 'auslesedatum'])

# Create 3D array with shape (num_vehicles, max_seq_len, dimension)
num_vehicles = output['VAN'].nunique()
max_seq_len_vector = input_vector.groupby('VAN').size().max()
dimension_vector = len(numerical_features)
X_vector = np.full((num_vehicles, max_seq_len_vector, dimension_vector), fill_value=SPECIAL_VALUE, dtype=np.float64)

# Scaling + Encoding
scaler = MinMaxScaler()
input_vector[numerical_features] = scaler.fit_transform(input_vector[numerical_features])

for van in input_vector['VAN'].unique():
    van_data = input_vector[input_vector['VAN'] == van]
    input_sequence = van_data.drop(columns=['VAN']).values
    index = output.index[output['VAN'] == van].to_list()[0]
    seq_len = input_sequence.shape[0]
    X_vector[index, 0:seq_len, :] = input_sequence

Rows before outlier removal: 95274
Rows after outlier removal: 74184


#### Split data into train and test set

In [424]:
train_size = int(0.8 * len(X_general))

X_vector_train, X_vector_test = X_vector[:train_size], X_vector[train_size:]
X_scalar_train, X_scalar_test = X_scalar[:train_size], X_scalar[train_size:]
X_general_train, X_general_test = X_general[:train_size], X_general[train_size:]

#### Create model

In [425]:
def custom_loss(y_true, y_pred, eps=1e-6):
    """Gaussian negative log likelihood loss function."""
    mean = y_pred[:, 0]
    log_var = y_pred[:, 1]
    var = tf.maximum(tf.exp(log_var), eps)
    loss = 0.5 * (tf.math.log(var) + tf.square(y_true - mean) / var)
    return tf.reduce_mean(loss)

def create_model(max_seq_len_vector, dimension_vector, max_seq_len_scalar, dimension_scalar, max_seq_len_info, dimension_info,
                 special_value, lstm_units=128, dense_units=128, num_lstm_layers=1, num_dense_hidden_layers=2, 
                 dropout_rate=0.4, transfer_learning_rate=0.001, activation='relu', transfer_learning=False):
    input_layer_vector = Input(name="input_vector", shape=(max_seq_len_vector, dimension_vector))
    input_layer_scalar = Input(name="input_scalar", shape=(max_seq_len_scalar, dimension_scalar))
    input_layer_general = Input(name="input_general", shape=(dimension_info,))

    masked_vector = Masking(name="masking_input_vector", mask_value=special_value)(input_layer_vector)
    masked_scalar = Masking(name="masking_input_scalar", mask_value=special_value)(input_layer_scalar)

    # Create LSTM layers with the specified number of units and layers
    lstm_output_vector = masked_vector
    lstm_output_scalar = masked_scalar

    for i in range(num_lstm_layers):
        lstm_output_vector = LSTM(lstm_units, name=f"lstm_vector_{i}", return_sequences=True if i < num_lstm_layers - 1 else False)(lstm_output_vector)
        lstm_output_scalar = LSTM(lstm_units, name=f"lstm_scalar_{i}", return_sequences=True if i < num_lstm_layers - 1 else False)(lstm_output_scalar)
        if dropout_rate > 0:
            lstm_output_vector = Dropout(dropout_rate, name=f"dropout_vector_{i}")(lstm_output_vector)
            lstm_output_scalar = Dropout(dropout_rate, name=f"dropout_scalar_{i}")(lstm_output_scalar)

    concat = concatenate([lstm_output_vector, lstm_output_scalar, input_layer_general])

    # Create dense layers with the specified number of units and layers
    dense_output = concat
    for i in range(num_dense_hidden_layers):
        dense_output = Dense(dense_units, activation=activation, name=f"dense_{i}")(dense_output)
        if dropout_rate > 0:
            dense_output = Dropout(dropout_rate, name=f"dropout_dense_{i}")(dense_output)

    mean_output = Dense(1, name="mean", activation=activation)(dense_output)
    log_var_output = Dense(1, name="log_var")(dense_output)
    output = concatenate([mean_output, log_var_output])

    model = Model(inputs=[input_layer_vector, input_layer_scalar, input_layer_general], outputs=output)
    
    optimizer = Adam(transfer_learning_rate if transfer_learning else 0.001)
    
    model.compile(optimizer=optimizer, loss=custom_loss)

    return model

#### Create model for transfer learning

In [426]:
def create_model_transfer_learning(base_model, max_seq_len_vector, dimension_vector, max_seq_len_scalar, dimension_scalar, max_seq_len_info, dimension_info,
                 special_value, lstm_units=128, dense_units=128, num_lstm_layers=1, num_dense_hidden_layers=2, 
                 dropout_rate=0.4, transfer_learning_rate=0.001, activation='relu'):
    new_model = create_model(max_seq_len_vector, dimension_vector, max_seq_len_scalar, dimension_scalar, max_seq_len_info, dimension_info,
                             special_value, lstm_units, dense_units, num_lstm_layers, num_dense_hidden_layers, 
                             dropout_rate, transfer_learning_rate, activation, transfer_learning=True)
    
    # Load pretrained weights
    for i in range(num_lstm_layers):
        new_model.get_layer(name=f"lstm_vector_{i}").set_weights(base_model.get_layer(name=f"lstm_vector_{i}").get_weights())

    for i in range(num_dense_hidden_layers):
        new_model.get_layer(name=f"dense_{i}").set_weights(base_model.get_layer(name=f"dense_{i}").get_weights())
    
    return new_model

#### Hyperparameter configuration

In [427]:
model_hyperparameter = {
    "lstm_units": 64, 
    "dense_units": 64, 
    "num_lstm_layers": 1, 
    "num_dense_hidden_layers": 2,
    "dropout_rate": 0.4, 
    "transfer_learning_rate": 0.0005, 
    "activation": 'relu', 
}

train_hyperparameter = {
    "normal_epochs": 1,
    "transfer_epochs": 1,
    "batch_size": 32
}

transfer_beta = False

#### Train Monte Carlo Dropout for all parameters with transfer learning

In [428]:
parameters = ['alpha_PE', 'alpha_NE', 'beta_PE', 'beta_NE']

models = dict()

for i, parameter in enumerate(parameters):
    y = output[[parameter]].values
    y_train = y[:train_size]
    pretrained_model = None
    if parameter == 'alpha_NE':
        pretrained_model = models['alpha_PE']
    if parameter == 'beta_PE' and transfer_beta:
        pretrained_model = models['alpha_NE']
    if parameter == 'beta_NE':
        pretrained_model = models['beta_PE']
    
    if pretrained_model is not None:
        model = create_model_transfer_learning(pretrained_model, max_seq_len_vector, dimension_vector, max_seq_len_scalar, dimension_scalar, max_seq_len_info, dimension_info, SPECIAL_VALUE, **model_hyperparameter)
    else:
        model = create_model(max_seq_len_vector, dimension_vector, max_seq_len_scalar, dimension_scalar, max_seq_len_info, dimension_info, SPECIAL_VALUE, **model_hyperparameter)

    model.fit(
        [X_vector_train, X_scalar_train, X_general_train],
        y_train,
        epochs=train_hyperparameter['transfer_epochs'] if pretrained_model is not None else train_hyperparameter['normal_epochs'], 
        batch_size=train_hyperparameter['batch_size'],
        verbose=1
    )

    models[parameter] = model

41/41 ━━━━━━━━━━━━━━━━━━━━ 20s 335ms/step - loss: -0.0904
41/41 ━━━━━━━━━━━━━━━━━━━━ 21s 329ms/step - loss: 0.1298
41/41 ━━━━━━━━━━━━━━━━━━━━ 18s 336ms/step - loss: -0.7166
41/41 ━━━━━━━━━━━━━━━━━━━━ 24s 473ms/step - loss: -0.5660


#### Evaluate Monte Carlo Droput for all parameters

In [429]:
def predict_with_dropout(model, inputs, n_iter=100):
    means = np.zeros((n_iter, len(inputs[0])))
    log_variances = np.zeros((n_iter, len(inputs[0])))

    for i in range(n_iter):
        preds = model(inputs, training=True)
        means[i] = preds[:,0]
        log_variances[i] = preds[:,1]
        
    return means, log_variances

In [430]:
def adjust_bound(lower_bound, upper_bound, min_val, max_val, z_score, std):
    for i in range(len(lower_bound)):
        if min_val < lower_bound[i] and max_val > upper_bound[i]:
            continue  # Case 1
        elif lower_bound[i] < min_val and max_val > upper_bound[i]:
            lower_bound[i] = min_val  # Case 2
        elif min_val < lower_bound[i] and upper_bound[i] > max_val:
            upper_bound[i] = max_val  # Case 3
        elif lower_bound[i] < min_val and upper_bound[i] > max_val:
            lower_bound[i] = min_val
            upper_bound[i] = max_val  # Case 4
        elif lower_bound[i] > max_val:
            upper_bound[i] = max_val
            lower_bound[i] = max(min_val, max_val - z_score * std[i])  # Case 5
        elif upper_bound[i] < min_val:
            lower_bound[i] = min_val
            upper_bound[i] = min(max_val, min_val + z_score * std[i])  # Case 6
            
    return lower_bound, upper_bound

In [431]:
def evaluate_mcd(model, X_vector_test, X_scalar_test, X_general_test, y_test, z_score=1.96, n_iter=2, aleatoric=True):
    results = {}
    
    # Get predictions with dropout
    means, log_variances = predict_with_dropout(model, [X_vector_test, X_scalar_test, X_general_test], n_iter=n_iter)
    
    max_val = np.max(y_test)
    min_val = np.min(y_test)

    # Calculate mean and variance according to the provided formula
    pred_mean = means.mean(axis=0)

    pred_epistemic_var = means.var(axis=0)
    pred_aleatoric_var = np.exp(log_variances).mean(axis=0)
    total_variance = pred_epistemic_var + (pred_aleatoric_var if aleatoric else 0)
    pred_std = np.sqrt(total_variance)
    pred_std_epistemic = np.sqrt(pred_epistemic_var)
    pred_std_aleatoric = np.sqrt(pred_aleatoric_var)
    
    lower_bound = pred_mean - z_score * pred_std
    upper_bound = pred_mean + z_score * pred_std
    
    lower_bound_epistemic = pred_mean - z_score * pred_std_epistemic
    upper_bound_epistemic = pred_mean + z_score * pred_std_epistemic
    
    lower_bound_aleatoric = pred_mean - z_score * pred_std_aleatoric
    upper_bound_aleatoric = pred_mean + z_score * pred_std_aleatoric
        
    # Total (unadjusted)
    covered = np.sum((y_test.flatten() >= lower_bound.flatten()) & (y_test.flatten() <= upper_bound.flatten()))
    picp = covered / len(y_test)
    mpiw = np.mean(np.abs(upper_bound - lower_bound), axis=0)
    target_range = max_val - min_val
    nmpiw = mpiw / target_range
    
    lower_bound_adj, upper_bound_adj = adjust_bound(lower_bound, upper_bound, min_val, max_val, z_score, pred_std)
    
    # Total (adjusted)
    covered_adj = np.sum((y_test.flatten() >= lower_bound_adj.flatten()) & (y_test.flatten() <= upper_bound_adj.flatten()))
    picp_adj = covered_adj / len(y_test)
    mpiw_adj = np.mean(upper_bound_adj - lower_bound_adj, axis=0)
    nmpiw_adj = mpiw_adj / target_range
    
    # Epistemic
    covered_epistemic = np.sum((y_test.flatten() >= lower_bound_epistemic.flatten()) & (y_test.flatten() <= upper_bound_epistemic.flatten()))
    picp_epistemic = covered_epistemic / len(y_test)
    mpiw_epistemic = np.mean(upper_bound_epistemic - lower_bound_epistemic, axis=0)
    nmpiw_epistemic = mpiw_epistemic / target_range
    
    # Aleatoric
    covered_aleatoric = np.sum((y_test.flatten() >= lower_bound_aleatoric.flatten()) & (y_test.flatten() <= upper_bound_aleatoric.flatten()))
    picp_aleatoric = covered_aleatoric / len(y_test)
    mpiw_aleatoric = np.mean(upper_bound_aleatoric - lower_bound_aleatoric, axis=0)
    nmpiw_aleatoric = mpiw_aleatoric / target_range
    
    mae = mean_absolute_error(y_test, pred_mean)
    mape = mean_absolute_percentage_error(y_test, pred_mean)

    results = {
        'mae': mae,
        'mape': mape,
        'picp': {
            'epistemic': picp_epistemic,
            'aleatoric': picp_aleatoric,
            'total': picp,
            'total_adj': picp_adj
        },
        'mpiw': {
            'epistemic': mpiw_epistemic,
            'aleatoric': mpiw_aleatoric,
            'total': mpiw,
            'total_adj': mpiw_adj
        },
        'nmpiw': {
            'epistemic': nmpiw_epistemic,
            'aleatoric': nmpiw_aleatoric,
            'total': nmpiw,
            'total_adj': nmpiw_adj,
        },
    }

    return results

In [432]:
z_score=1.96
n_iter=2
aleatoric=True

results = {}

for parameter in ['alpha_PE', 'alpha_NE', 'beta_PE', 'beta_NE']:
    y = output[[parameter]].values
    train_size = int(0.8 * len(y))
    y_test = y[train_size:]
    model = models[parameter]
    results[parameter] = evaluate_mcd(model=model, X_general_test=X_general_test, X_scalar_test=X_scalar_test, X_vector_test=X_vector_test, y_test=y_test, n_iter=n_iter, aleatoric=aleatoric, z_score=z_score)

In [433]:
trial = {
    'model_hyperparameter': model_hyperparameter,
    'train_hyperparameter': train_hyperparameter,
    'results': results
}

def convert_numpy_types(obj):
    if isinstance(obj, np.float32):
        return float(obj)
    elif isinstance(obj, np.int32):
        return int(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Load the existing JSON data from file
try:
    with open('monte_carlo_dropout_results.json', 'r') as json_file:
        data = json.load(json_file)
except FileNotFoundError:
    data = []

# Append the new trial
data.append(trial)

# Save the updated data back to the JSON file
with open('monte_carlo_dropout_results.json', 'w') as json_file:
    json.dump(data, json_file, indent=4, default=convert_numpy_types)